In [1]:
import ase.io
from pet.hypers import load_hypers_from_file
from pet.data_preparation import get_all_species
from pet.pet import PET, PETUtilityWrapper, PETMLIPWrapper
import torch
from pet.molecule import MoleculeCPP, Molecule
from matscipy.neighbours import neighbour_list as neighbor_list
torch.set_default_dtype(torch.float32)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def prepare_test(stucture_path, r_cut, n_gnn, n_trans, hypers_path = "../default_hypers/default_hypers.yaml"):
    structure = ase.io.read(stucture_path, index=0)
    hypers = load_hypers_from_file(hypers_path)
    

    MLIP_SETTINGS = hypers.MLIP_SETTINGS
    ARCHITECTURAL_HYPERS = hypers.ARCHITECTURAL_HYPERS
    FITTING_SCHEME = hypers.FITTING_SCHEME

    ARCHITECTURAL_HYPERS.D_OUTPUT = 1  # energy is a single scalar
    ARCHITECTURAL_HYPERS.TARGET_TYPE = "structural"  # energy is structural property
    ARCHITECTURAL_HYPERS.TARGET_AGGREGATION = (
        "sum"  # energy is a sum of atomic energies
    )
    ARCHITECTURAL_HYPERS.R_CUT = r_cut
    ARCHITECTURAL_HYPERS.N_TRANS_LAYERS = n_trans
    ARCHITECTURAL_HYPERS.N_GNN_LAYERS = n_gnn
    all_species = get_all_species([structure])


    model = PET(ARCHITECTURAL_HYPERS, 0.0, len(all_species)).to(device)
    model = PETUtilityWrapper(model, FITTING_SCHEME.GLOBAL_AUG)

    model = PETMLIPWrapper(
        model, MLIP_SETTINGS.USE_ENERGIES, MLIP_SETTINGS.USE_FORCES
    )
    return model, structure, all_species, ARCHITECTURAL_HYPERS





In [3]:
def get_predictions_old_python(model, structure, all_species, ARCHITECTURAL_HYPERS):
    
    molecule = Molecule(
        structure,
        ARCHITECTURAL_HYPERS.R_CUT,
        ARCHITECTURAL_HYPERS.USE_ADDITIONAL_SCALAR_ATTRIBUTES,
        ARCHITECTURAL_HYPERS.USE_LONG_RANGE,
        ARCHITECTURAL_HYPERS.K_CUT,
    )
    if ARCHITECTURAL_HYPERS.USE_LONG_RANGE:
        raise NotImplementedError(
            "Long range interactions are not supported in the SingleStructCalculator"
        )

    graph = molecule.get_graph(
        molecule.get_max_num(), all_species, None
    )
    graph.batch = torch.zeros(
        graph.num_nodes, dtype=torch.long, device=graph.x.device
    )
    graph = graph.to(device)
    prediction_energy, prediction_forces = model(
        graph, augmentation=False, create_graph=False
    )

    return prediction_energy, prediction_forces, graph

def get_predictions_cpp(model, structure, all_species, ARCHITECTURAL_HYPERS):
    
    molecule = MoleculeCPP(
        structure,
        ARCHITECTURAL_HYPERS.R_CUT,
        ARCHITECTURAL_HYPERS.USE_ADDITIONAL_SCALAR_ATTRIBUTES,
        ARCHITECTURAL_HYPERS.USE_LONG_RANGE,
        ARCHITECTURAL_HYPERS.K_CUT,
    )
    if ARCHITECTURAL_HYPERS.USE_LONG_RANGE:
        raise NotImplementedError(
            "Long range interactions are not supported in the SingleStructCalculator"
        )

    graph = molecule.get_graph(
        molecule.get_max_num(), all_species, None
    )
    graph.batch = torch.zeros(
        graph.num_nodes, dtype=torch.long, device=graph.x.device
    )
    graph = graph.to(device)
    prediction_energy, prediction_forces = model(
        graph, augmentation=False, create_graph=False
    )

    return prediction_energy, prediction_forces, graph

In [4]:
model, structure, all_species, ARCHITECTURAL_HYPERS = prepare_test("../example/methane_train.xyz", 10.0, 2, 2)
python_energy, python_forces, python_graph = get_predictions_old_python(model, structure, all_species, ARCHITECTURAL_HYPERS)
cpp_energy, cpp_forces, cpp_graph = get_predictions_cpp(model, structure, all_species, ARCHITECTURAL_HYPERS)

print("Energy difference: ", torch.abs(python_energy - cpp_energy))
print("Forces difference: ", torch.abs(python_forces - cpp_forces).max())

Energy difference:  tensor([0.], grad_fn=<AbsBackward0>)
Forces difference:  tensor(0.)


In [5]:
model, structure, all_species, ARCHITECTURAL_HYPERS = prepare_test("bulk.xyz", 4.0, 2, 2)
python_energy, python_forces, python_graph = get_predictions_old_python(model, structure, all_species, ARCHITECTURAL_HYPERS)
cpp_energy, cpp_forces, cpp_graph = get_predictions_cpp(model, structure, all_species, ARCHITECTURAL_HYPERS)

print("Energy difference: ", torch.abs(python_energy - cpp_energy))
print("Forces difference: ", torch.abs(python_forces - cpp_forces).max())

print("Forces spread: ", torch.abs(python_forces).max())

Energy difference:  tensor([0.0002], grad_fn=<AbsBackward0>)
Forces difference:  tensor(1.9222e-06)
Forces spread:  tensor(3.6096)


In [6]:
%timeit get_predictions_old_python(model, structure, all_species, ARCHITECTURAL_HYPERS)

2.53 s ± 218 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit get_predictions_cpp(model, structure, all_species, ARCHITECTURAL_HYPERS)

2.16 s ± 128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit i_list, j_list, D_list, S_list = neighbor_list('ijDS', structure, 4.0)

5.42 ms ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit i_list, j_list, D_list, S_list = ase.neighborlist.neighbor_list("ijDS", structure, 4.0)

91.1 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
 def is_3d_crystal(atoms):
    pbc = atoms.get_pbc()
    if isinstance(pbc, bool):
        return pbc
    return all(pbc)

print(is_3d_crystal(structure))

True
